First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [11]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(vocab_size)

65


In [12]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:"".join([itos[j] for j in l])
print(encode("Hello World"))
print(decode(encode("Hello World")))


[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [13]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape)

torch.Size([1115394])


In [14]:
n=int(0.9*len(data))
print(n)
train_data=data[:n]
test_data=data[n:]

1003854


In [15]:
from torch import tensor

block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for i in range(block_size):
    context=x[:i+1]
    target=y[i]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [17]:
torch.manual_seed(1337)
batch_size=4
block_size=8
def get_batch(split):
    data=train_data if split=="train" else test_data
    ix=torch.randint(len(data)-block_size,(batch_size,1))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
xb,yb=get_batch("train")
print(xb.shape)
print(yb.shape)
for b in range(batch_size):
    for t in range(block_size):
        context=xb[b,:t+1]
        target=yb[b,t]
        print(f"when input is {context} the target is {target}")




torch.Size([4, 8])
torch.Size([4, 8])
when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the target is 1
when input is tensor([44, 53, 56,  1]) the target is 58
when input is tensor([44, 53, 56,  1, 58]) the target is 46
when input is tensor([44, 53, 56,  1, 58, 46]) the target is 39
when input is tensor([44, 53, 56,  1, 58, 46, 39]) the target is 58
when input is tensor([44, 53, 56,  1, 58, 46, 39, 58]) the target is 1
when input is tensor([52]) the target 

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx)
        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss=self(idx)
            logits=logits[:,-1,:]
            probs=F.softmax(logits,dim=-1)
            idx_next=torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx,idx_next),dim=1)
        return idx


m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [30]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [44]:
batch_size=32
for steps in range(10000):
    xb,yb=get_batch("train")
    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


2.599977970123291


In [45]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


ARithey aranoseno th p thends?
my wr, od alllo t nathed ou od stoure caricomo s hirelolinof f d cesplad wacoussare'd sous an?
IO:
Busen
ANGarenokeceor okitldlof m se HA:
B-
y y so of y a t imyowaloudeatoreve oug;
Ove be g s, s istoticet whed brter wirserille f towort y
Then
ber twhe'
Anoup ban'RINour:
Ar ld s che at IO: wrous st
BELO hinghe inor ile GLARDUS:

IOREd Cocoif me INI thist p tit f wikn t d s lom?
Th peaspids boke exth thodoueall t lt, d d sely frbed by

GLatee IINoursthensho-d CAnd,

